### API key 작동하는지 확인 
* 여기서 decoding 된 key 사용했음

In [ ]:
import requests
mykey = 'slI57RuOt8oShUuGo9+6bKJevroNLwAkgqM/PAPuRK9pB6v2E8BKxoiiFBzINP4vTDlkzy4zYnSh4kygjc9puQ=='
url = 'http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getMsrstnAcctoRltmMesureDnsty'
params ={'serviceKey' : mykey, 
         'returnType' : 'json', 
         'numOfRows' : '2159', 
         'pageNo' : '1', 
         'stationName':'종로구',
         'dataTerm':'3MONTH',
         'ver':'1.0'}

response = requests.get(url, params=params)
print(response.content)

b'{"response":{"body":{"totalCount":2158,"items":[{"so2Grade":"1","coFlag":null,"khaiValue":"68","so2Value":"0.003","coValue":"0.4","pm25Flag":null,"pm10Flag":null,"pm10Value":"47","o3Grade":"2","khaiGrade":"2","pm25Value":"19","no2Flag":null,"no2Grade":"1","o3Flag":null,"pm25Grade":"2","so2Flag":null,"dataTime":"2022-03-27 14:00","coGrade":"1","no2Value":"0.008","pm10Grade":"2","o3Value":"0.052"},{"so2Grade":"1","coFlag":null,"khaiValue":"64","so2Value":"0.004","coValue":"0.4","pm25Flag":null,"pm10Flag":null,"pm10Value":"39","o3Grade":"2","khaiGrade":"2","pm25Value":"12","no2Flag":null,"no2Grade":"1","o3Flag":null,"pm25Grade":"1","so2Flag":null,"dataTime":"2022-03-27 13:00","coGrade":"1","no2Value":"0.009","pm10Grade":"2","o3Value":"0.047"},{"so2Grade":"1","coFlag":null,"khaiValue":"63","so2Value":"0.004","coValue":"0.4","pm25Flag":null,"pm10Flag":null,"pm10Value":"33","o3Grade":"2","khaiGrade":"2","pm25Value":"16","no2Flag":null,"no2Grade":"1","o3Flag":null,"pm25Grade":"1","so2Flag":

### API json 형태로 불러오기

In [ ]:
import urllib.request
import json
from pandas import json_normalize
from urllib.parse import urlencode, unquote, quote_plus 

In [ ]:
api = 'http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getMsrstnAcctoRltmMesureDnsty' # 서비스 url
key = unquote("slI57RuOt8oShUuGo9+6bKJevroNLwAkgqM/PAPuRK9pB6v2E8BKxoiiFBzINP4vTDlkzy4zYnSh4kygjc9puQ==")

In [ ]:
queryParams = '?' + urlencode({quote_plus('serviceKey') : key, 
                               quote_plus('returnType') : 'json', 
                               quote_plus('numOfRows') : '2159', 
                               quote_plus('pageNo') : '1', 
                               quote_plus('stationName'):'강남구',
                               quote_plus('dataTerm'):'3MONTH',
                               quote_plus('ver'):'1.0' })

url = api + queryParams

In [ ]:
text = urllib.request.urlopen(url).read().decode('utf-8')

In [1]:
json_return = json.loads(text)
get_data = json_return.get('response')

NameError: ignored

In [ ]:
df = json_normalize(get_data['body']['items'])
df

### 데이터 가공

In [ ]:
df.columns

In [ ]:
df = df.drop(['so2Grade', 'coFlag','so2Value', 'coValue', 'o3Grade','no2Flag','no2Grade', 'o3Flag', 'so2Flag','coGrade','no2Value','o3Value'], axis=1)

In [ ]:
df = df.reindex(columns=['dataTime', 'khaiValue','khaiGrade','pm25Value','pm25Grade','pm25Flag','pm10Value','pm10Grade','pm10Flag'])
df

### csv로 저장

In [ ]:
df.to_csv("micro-dust-강남구.csv")

### 반복문으로 구 별 데이터 csv 저장하기

In [ ]:
district = ['강남구','강동구','강북구','강서구','관악구',
            '광진구','구로구','금천구','노원구','도봉구',
            '동대문구','동작구','마포구','서대문구','서초구',
            '성동구','성북구','송파구','양천구','영등포구',
            '용산구','은평구','종로구','중구', '중랑구']

In [ ]:
test = ['강남구']
api = 'http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getMsrstnAcctoRltmMesureDnsty' # 서비스 url
key = unquote("slI57RuOt8oShUuGo9+6bKJevroNLwAkgqM/PAPuRK9pB6v2E8BKxoiiFBzINP4vTDlkzy4zYnSh4kygjc9puQ==")

In [ ]:
for i in test:
  # 구 별 api 만들기 
  queryParams = '?' + urlencode({quote_plus('serviceKey') : key, 
                               quote_plus('returnType') : 'json', 
                               quote_plus('numOfRows') : '2159', 
                               quote_plus('pageNo') : '1', 
                               quote_plus('stationName'):i,
                               quote_plus('dataTerm'):'3MONTH',
                               quote_plus('ver'):'1.1' }) # 24시간 예측 평균 이동 값 있음

  url = api + queryParams

  # json 불러와서 data frame으로 저장
  text = urllib.request.urlopen(url).read().decode('utf-8')
  json_return = json.loads(text)
  get_data = json_return.get('response')
  df = json_normalize(get_data['body']['items'])

  # 데이터 가공
  # df = df.drop(['so2Grade', 'coFlag','so2Value', 'coValue', 'o3Grade','no2Flag','no2Grade','pm25Flag', 'o3Flag','pm10Flag', 'so2Flag','coGrade','no2Value','o3Value'], axis=1)
  # df = df.reindex(columns=['dataTime', 'khaiValue','khaiGrade','pm25Value','pm25Grade','pm10Value','pm10Grade'])

  # csv 저장하기
  df.to_csv("test"+str(i)+".csv")


### 일 평균 구하기

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("test강남구.csv")
data.head()


,Unnamed: 0,so2Grade,coFlag,pm10Value24,khaiValue,so2Value,coValue,pm25Flag,pm10Flag,pm10Value,...,no2Flag,no2Grade,o3Flag,pm25Grade,so2Flag,dataTime,coGrade,no2Value,pm10Grade,o3Value
0,0,1.0,NaN,24,59,0.004,0.5,NaN,NaN,29,...,NaN,2.0,NaN,1.0,NaN,2022-03-28 10:00,1.0,0.036,1.0,0.015
1,1,1.0,NaN,23,68,0.003,0.5,NaN,NaN,29,...,NaN,2.0,NaN,1.0,NaN,2022-03-28 09:00,1.0,0.041,1.0,0.010
2,2,1.0,NaN,21,71,0.003,0.5,NaN,NaN,26,...,NaN,2.0,NaN,1.0,NaN,2022-03-28 08:00,1.0,0.043,1.0,0.005
3,3,1.0,NaN,20,70,0.003,0.4,NaN,NaN,22,...,NaN,2.0,NaN,1.0,NaN,2022-03-28 07:00,1.0,0.042,1.0,0.005
4,4,1.0,NaN,20,51,0.003,0.4,NaN,NaN,21,...,NaN,2.0,NaN,1.0,NaN,2022-03-28 06:00,1.0,0.031,1.0,0.012
